In [1]:
!pip install earth2studio[dlwp] earth2studio[pangu] earth2studio[sfno] earth2studio[cyclone] "makani @ git+https://github.com/NVIDIA/modulus-makani.git@28f38e3e929ed1303476518552c64673bbd6f722"

  Using cached makani-0.2.0-py3-none-any.whl
  Attempting uninstall: makani
    Found existing installation: makani 0.2.0
    Uninstalling makani-0.2.0:
      Successfully uninstalled makani-0.2.0


In [2]:
!git clone https://github.com/NVIDIA/makani.git
%cd makani
!ls
!pip install -e .

fatal: destination path 'makani' already exists and is not an empty directory.
/content/makani
config	      docker   makani.egg-info	README.md	     tests
cufile.log    images   Makefile		sfno_output10.nc
data_process  LICENSE  notebooks	sfno_output.nc
datasets      makani   pyproject.toml	sfno_tc_tracks.zarr
Obtaining file:///content/makani
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for makani (pyproject.toml) ... done
  Created wheel for makani: filename=makani-0.2.0-0.editable-py3-none-any.whl size=11944 sha256=f4eec394d41a9b790b783e91543e75dee6ecef6328690153f6da7d8ffa2650b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-8f5_xpi3/wheels/41/08/cc/e35c7893235a2486ffbb8a10403dcaca864cebc516904fd972
Successfully built makani
  Attempting uninstall: makani
    Found existing installation: makani 0.2.0
  

In [3]:
import numpy as np
import torch
from datetime import datetime

from earth2studio.models.px import SFNO
from earth2studio.data import GFS, fetch_data
from earth2studio.utils.time import to_time_array
from earth2studio.utils.coords import map_coords
from earth2studio.models.dx import TCTrackerVitart

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


CuPy distance computation test failed with error: cuVS >= 24.12 or pylibraft < 24.12 should be installed to use this feature


In [4]:
# import torch
# from earth2studio.models.px import Pangu3
# from earth2studio.data import GFS
# from earth2studio.io import NetCDF4Backend
# from earth2studio.run import deterministic as run

# # 1. Pick a device
# if not torch.cuda.is_available():
#     raise RuntimeError("CUDA not available – check your WSL2/NVIDIA setup")

# device = torch.device("cuda:0")

# # 2. Load Pangu and move it to GPU
# model = Pangu3.load_model(Pangu3.load_default_package())
# model = model.to(device)  # this also rebuilds the ONNX Runtime session on CUDA

# # 3. Data source and IO
# ds = GFS()
# io = NetCDF4Backend("output.nc")

# start_times = ["2024-01-01T00:00"]
# steps = 1

# # 4. Run forecast on that device
# run(start_times, steps, model, ds, io, device=device)


In [7]:
import torch
from earth2studio.models.px import SFNO
from earth2studio.data import GFS
from earth2studio.io import NetCDF4Backend
from earth2studio.run import deterministic as run
from earth2studio.run import diagnostic
from earth2studio.io import ZarrBackend

# 1. Pick a device
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available – check your WSL2/NVIDIA setup")

device = torch.device("cuda:0")

# 2. Load SFNO (FourCastNet-style) and move it to GPU
model = SFNO.load_model(SFNO.load_default_package())
model = model.to(device)

# 3. Diagnostic model: TC tracker
tc_model = TCTrackerVitart().to(device)

# 3. Data source and IO
ds = GFS()
io = ZarrBackend("/content/sfno_tc_tracks.zarr")

# SFNO is a 6-hour timestep model:
# steps = 1 → +6 h, steps = 4 → +24 h, etc.
start_times = ["2024-01-01T00:00"]
steps = 10

# 4. Run forecast on that device
run(start_times, steps, model, ds, io, device=device)


2025-12-04 00:40:03.959 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-12-04 00:40:03.959 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda:0


Fetching GFS data:   0%|          | 0/73 [00:00<?, ?it/s]

2025-12-04 00:40:03.968 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 170010298-904882
2025-12-04 00:40:04.009 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 354558976-950206
2025-12-04 00:40:04.050 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 194463909-743465
2025-12-04 00:40:04.091 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 268038708-839499
2025-12-04 00:40:04.131 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 165935718-1138677


Fetching GFS data:   0%|          | 0/73 [00:00<?, ?it/s]

2025-12-04 00:40:04.173 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 350072351-1232650
2025-12-04 00:40:04.214 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 452597070-961302
2025-12-04 00:40:04.255 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 210217024-602993
2025-12-04 00:40:04.299 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 180172685-913532
2025-12-04 00:40:04.340 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 387322476-948106


...

2025-12-04 00:40:06.526 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 292088625-1244938
2025-12-04 00:40:06.566 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 407591936-940269
2025-12-04 00:40:06.605 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 189450163-577835
2025-12-04 00:40:06.645 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 149106074-462399
2025-12-04 00:40:06.684 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 330670600-938837
2025-12-04 00:40:06.723 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS

Fetching GFS data: 100%|██████████| 73/73 [00:02<00:00, 24.71it/s]


2025-12-04 00:40:06.763 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 145318799-1019625
2025-12-04 00:40:06.803 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 326166249-1225442
2025-12-04 00:40:06.842 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 451628742-968328
2025-12-04 00:40:06.881 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240101/00/atmos/gfs.t00z.pgrb2.0p25.f000 199346060-588823
2025-12-04 00:40:07.302 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from GFS
2025-12-04 00:40:07.501 | INFO     | earth2studio.run:deterministic:136 - Inference starting!



Running inference: 100%|██████████| 11/11 [02:43<00:00, 14.85s/it]

2025-12-04 00:42:50.875 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete


In [ ]:
# Model's required coordinate system
prognostic_ic = model.input_coords()

time = to_time_array(start_times)

x0, coords0 = fetch_data(
    source=ds,
    time=time,
    variable=prognostic_ic["variable"],
    lead_time=prognostic_ic["lead_time"],
    device=device,
    interp_to=prognostic_ic if hasattr(model, "interp_method") else None,
    interp_method=getattr(model, "interp_method", "nearest"),
)

# Make sure coords/x are in the layout SFNO expects
x0, coords0 = map_coords(x0, coords0, prognostic_ic)

print("x0 shape:", x0.shape)
print("coords0 keys:", list(coords0.keys()))
for k, v in coords0.items():
    print(k, v.shape, v.dtype)



In [ ]:
def integrate(model, x_init, coords_init, nsteps: int):
    """
    Returns list of tensors [x^0, x^1, ..., x^nsteps] on CPU,
    plus the coords for the last step.
    """
    it = model.create_iterator(x_init, coords_init)
    out = []
    for step, (x_step, coords_step) in enumerate(it):
        out.append(x_step.detach().cpu())
        if step == nsteps:
            break
    return out, coords_step


In [ ]:
x_ctrl_init = x0.clone()
ctrl_steps, coords_final = integrate(model, x_ctrl_init, coords0, steps)


In [ ]:
# Figure out which axis is which
dim_names = list(coords0.keys())      # e.g. ["time", "lead_time", "variable", "lat", "lon"]
axis = {name: i for i, name in enumerate(dim_names)}

var_names = np.array(coords0["variable"])
lat = np.array(coords0["lat"])
lon = np.array(coords0["lon"])

# Choose variable and region to perturb
target_var = "t850"        # or "t2m", etc.
dT = 2.0                   # K

lat_min, lat_max = -10, 10   # example box (tropics)
lon_min, lon_max =  60, 120  # example box (Indian Ocean-ish)

var_idx = int(np.where(var_names == target_var)[0][0])
lat_mask = (lat >= lat_min) & (lat <= lat_max)
lon_mask = (lon >= lon_min) & (lon <= lon_max)

print("Perturbing", target_var, "in",
      f"lat[{lat_min},{lat_max}], lon[{lon_min},{lon_max}]")

# Move initial condition to CPU/NumPy for easy slicing
x_base_np = x0.detach().cpu().numpy()

# Integer indices for the patch
lat_idx = np.where(lat_mask)[0]
lon_idx = np.where(lon_mask)[0]

# Use np.ix_ to get a 2D grid of indices that broadcast correctly
lat_ix, lon_ix = np.ix_(lat_idx, lon_idx)

# Build index tuple matching x_base_np's dimensions
idx = [slice(None)] * x_base_np.ndim
idx[axis["variable"]] = var_idx   # scalar is fine
idx[axis["lat"]] = lat_ix         # shape (nlat, 1)
idx[axis["lon"]] = lon_ix         # shape (1, nlon)

x_pert_np = x_base_np.copy()
x_pert_np[tuple(idx)] += dT   # now adds +dT on the full (lat, lon) box

# Back to torch on GPU
x_pert = torch.from_numpy(x_pert_np).to(device)


In [ ]:
pert_steps, coords_final_pert = integrate(model, x_pert, coords0, steps)


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# 1. Stack lists of tensors into one big array
ctrl_np = np.stack([t.numpy() for t in ctrl_steps], axis=0)   # (step, ...)
pert_np = np.stack([t.numpy() for t in pert_steps], axis=0)   # (step, ...)

n_steps_plus_1 = ctrl_np.shape[0]

# 2. Figure out data dims from coords0 (same as initial coords)
dim_names = list(coords0.keys())          # e.g. ["time","lead_time","variable","lat","lon"]

coords_for_xr = {}
for name in dim_names:
    coords_for_xr[name] = np.array(coords0[name])

# Add a "step" dimension and a convenient lead time coordinate in hours
step_vals = np.arange(n_steps_plus_1)
lead_hours = 6 * step_vals   # SFNO is 6-h timestep

coords_full = {
    "step": step_vals,
    "lead_hours": ("step", lead_hours),
}
coords_full.update(coords_for_xr)

# 3. Build DataArrays
x_ctrl = xr.DataArray(
    ctrl_np,
    dims=("step",) + tuple(dim_names),
    coords=coords_full,
    name="control",
)

x_pert = xr.DataArray(
    pert_np,
    dims=("step",) + tuple(dim_names),
    coords=coords_full,
    name="perturbed",
)

x_diff = x_pert - x_ctrl
x_diff.name = "pert_minus_ctrl"

target_var = "t850"  # same as before

# Select that variable and collapse time/lead_time (they're usually length 1)
ctrl_var = x_ctrl.sel(variable=target_var).isel(time=0, lead_time=0)
pert_var = x_pert.sel(variable=target_var).isel(time=0, lead_time=0)
diff_var = x_diff.sel(variable=target_var).isel(time=0, lead_time=0)
# shapes: (step, lat, lon)
rms_by_step = np.sqrt((diff_var ** 2).mean(dim=("lat", "lon")))

plt.figure(figsize=(8, 4))
rms_by_step.plot(x="lead_hours", marker="o")
plt.xlabel("Lead time (hours)")
plt.ylabel(f"RMS difference in {target_var}")
plt.title(f"Growth of perturbation in {target_var}")
plt.grid(True)
plt.show()
# Choose which steps to visualize (e.g. every 2nd step)
steps_to_show = range(0, n_steps_plus_1, 2)

for k in steps_to_show:
    ctrl_k = ctrl_var.sel(step=k)
    pert_k = pert_var.sel(step=k)
    diff_k = diff_var.sel(step=k)

    fig, axs = plt.subplots(1, 3, figsize=(15, 4), constrained_layout=True)

    ctrl_k.plot(ax=axs[0], x="lon", y="lat")
    axs[0].set_title(f"Control {target_var}, +{int(lead_hours[k])} h")

    pert_k.plot(ax=axs[1], x="lon", y="lat")
    axs[1].set_title(f"Perturbed {target_var}, +{int(lead_hours[k])} h")

    diff_k.plot(ax=axs[2], x="lon", y="lat")
    axs[2].set_title(f"Difference, +{int(lead_hours[k])} h")

    plt.show()
